## Purpose: This file works with the raw Qualtrics survey CSV data file obtained as of 11/01/2023 (with no identifying information kept), and performs initial data cleaning steps. Finally, an output file is generated with the cleaned data, which can then be used as an input file in Clustering.ipynb.

In [ ]:
# import dependencies
from google.colab import drive
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
# mount Google Drive to access files
# not needed if running the file locally
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive




---

# Grab data and divide it into first and second halves


---



In [ ]:
# raw data file
filepath = '/content/drive/Shareddrives/Key Fall 2023/Survey Data/Final code/11_01_final_raw_data.csv'
# read data
data = pd.read_csv(filepath)
data.head()

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,Q25,Q26,Q39,Q32,Q28,Q29,Q30,Q31,Q33,Q34
0,Start Date,End Date,Response Type,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Distribution Channel,User Language,...,"I have $1,300 in my Venmo account. If Venmo go...",I know I have resources to successfully manage...,"Where are you currently located? (City,State,C...",What is your current age?,At what age did you start managing your own fi...,What I know about money management today I mos...,"Growing up, my parents…","Growing up, about how much was your parents’ c...",What is your highest level of education comple...,What is your gender identity?
1,"{""ImportId"":""startDate"",""timeZone"":""America/Ne...","{""ImportId"":""endDate"",""timeZone"":""America/New_...","{""ImportId"":""status""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""distributionChannel""}","{""ImportId"":""userLanguage""}",...,"{""ImportId"":""QID26""}","{""ImportId"":""QID27""}","{""ImportId"":""QID42_TEXT""}","{""ImportId"":""QID34""}","{""ImportId"":""QID30""}","{""ImportId"":""QID31""}","{""ImportId"":""QID32""}","{""ImportId"":""QID33""}","{""ImportId"":""QID35""}","{""ImportId"":""QID36""}"
2,2023-10-08 17:05:16,2023-10-08 17:09:03,IP Address,100,227,TRUE,2023-10-08 17:09:04,R_3ekofckgob0t9tQ,anonymous,EN,...,I do not know.,Not really confident,NaN,19,15-17,My family,"Openly talked about money, but did not have ad...","45,000 - 75,000",1-2 years of college,Cis-woman
3,2023-10-08 17:24:02,2023-10-08 17:31:08,IP Address,100,426,TRUE,2023-10-08 17:31:09,R_1oA4iLuR3g55Xac,anonymous,EN,...,I do not know.,Very confident,"Champaign, Il, USA",20,15-17,My family,"Were very private about their money, but did h...","160,000 - 180,000",1-2 years of college,Cis-woman
4,2023-10-08 17:23:24,2023-10-08 18:33:35,IP Address,100,4210,TRUE,2023-10-08 18:33:36,R_C3LmwSIcGs0MvHr,anonymous,EN,...,Maybe.,Somewhat confident,"Gurnee, IL, USA",20,15-17,Independent research,"Were very private about their money, but did h...","45,000 - 75,000",1-2 years of college,Cis-woman


In [ ]:
# drop first 2 rows containing column headers and descriptions
data = data.drop([0,1])
data.head()

,StartDate,EndDate,Status,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,DistributionChannel,UserLanguage,...,Q25,Q26,Q39,Q32,Q28,Q29,Q30,Q31,Q33,Q34
2,2023-10-08 17:05:16,2023-10-08 17:09:03,IP Address,100,227,TRUE,2023-10-08 17:09:04,R_3ekofckgob0t9tQ,anonymous,EN,...,I do not know.,Not really confident,NaN,19,15-17,My family,"Openly talked about money, but did not have ad...","45,000 - 75,000",1-2 years of college,Cis-woman
3,2023-10-08 17:24:02,2023-10-08 17:31:08,IP Address,100,426,TRUE,2023-10-08 17:31:09,R_1oA4iLuR3g55Xac,anonymous,EN,...,I do not know.,Very confident,"Champaign, Il, USA",20,15-17,My family,"Were very private about their money, but did h...","160,000 - 180,000",1-2 years of college,Cis-woman
4,2023-10-08 17:23:24,2023-10-08 18:33:35,IP Address,100,4210,TRUE,2023-10-08 18:33:36,R_C3LmwSIcGs0MvHr,anonymous,EN,...,Maybe.,Somewhat confident,"Gurnee, IL, USA",20,15-17,Independent research,"Were very private about their money, but did h...","45,000 - 75,000",1-2 years of college,Cis-woman
5,2023-10-08 19:31:07,2023-10-08 19:43:36,IP Address,100,749,TRUE,2023-10-08 19:43:37,R_3n2uwLUj9MJvinV,anonymous,EN,...,I do not know.,Neutral,Chicago,20,15-17,My family,"Openly talked about money, but did not have ad...","< 45,000",3-4 years of college,Cis-man
6,2023-10-08 20:50:14,2023-10-08 21:01:13,IP Address,100,659,TRUE,2023-10-08 21:01:14,R_3JjS3XbzVuCbTns,anonymous,EN,...,Maybe.,Somewhat confident,"Gurnee, IL USA",20,15-17,My family,"Openly talked about money, and always had mone...",">180,000",1-2 years of college,Cis-woman


In [ ]:
# divide first and second halves of the data
data_1 = data.iloc[:, :40]
data_2 = data.iloc[:, 40:]
data_1.shape, data_2.shape

((177, 40), (177, 14))


---

# Cleaning the first half of the data

---

In [ ]:
# drop columns that are not useful
data_1 = data_1.drop(['StartDate', 'EndDate', 'Status', 'Progress', 'Duration (in seconds)', 'RecordedDate', 'ResponseId', 'DistributionChannel',
       'UserLanguage'], axis=1)
data_1.columns

Index(['Finished', 'Q2', 'Q2_10_TEXT', 'Q3', 'Q3_9_TEXT', 'Q4', 'Q5', 'Q6',
       'Q8', 'Q9', 'Q10', 'Q10_4_TEXT', 'Q11', 'Q12', 'Q13', 'Q14_1', 'Q14_2',
       'Q14_3', 'Q14_4', 'Q14_5', 'Q14_6', 'Q14_7', 'Q14_8', 'Q14_8_TEXT',
       'Q40', 'Q15', 'Q16', 'Q16_6_TEXT', 'Q17', 'Q18', 'Q19'],
      dtype='object')

In [ ]:
# encode categorical columns
encode_list = ['Finished', 'Q4', 'Q5', 'Q8', 'Q9', 'Q12', 'Q15']
encoder = LabelEncoder()
encodings = {}
for e in encode_list:
  data_1[e].values[:] = encoder.fit_transform(data[e])
  encodings[e] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

# dummies for Q2, Q3, Q10, Q40, Q16
q2_dummies = data_1['Q2'].str.get_dummies(sep=",").drop("Other (please specify)", axis=1).add_prefix('Q2_')
q3_dummies = data_1['Q3'].str.get_dummies(sep=",").drop("Other (please specify)", axis=1).add_prefix('Q3_')
q10_dummies = data_1['Q10'].str.get_dummies(sep=",").drop("Other (please specify)", axis=1).add_prefix('Q10_')
q40_dummies = data_1['Q40'].str.get_dummies(sep=",").add_prefix('Q40_')
q16_dummies = data_1['Q16'].str.get_dummies(sep=",").drop("Other (please specify)", axis=1).add_prefix('Q16_')

# drop original columns for dummies
data_1 = data_1.drop(['Q2', 'Q3', 'Q10', 'Q40', 'Q16'], axis=1)
data_1.head()

,Finished,Q2_10_TEXT,Q3_9_TEXT,Q4,Q5,Q6,Q8,Q9,Q10_4_TEXT,Q11,...,Q14_5,Q14_6,Q14_7,Q14_8,Q14_8_TEXT,Q15,Q16_6_TEXT,Q17,Q18,Q19
2,1,NaN,NaN,2,0,NaN,1,1,NaN,Getting a job and a flow of income,...,3,2,7,8,NaN,4,NaN,NaN,NaN,NaN
3,1,NaN,NaN,2,0,NaN,1,1,NaN,Often transferred extra money when needed from...,...,1,2,6,8,NaN,2,NaN,My dad who works in fiance,Nothing specifically,Ease of use and quality of UI/User Experience
4,1,NaN,NaN,2,0,NaN,1,1,NaN,Direct deposit for work,...,2,6,4,8,NaN,1,NaN,My parents,ATMs seem to be non functional very often and ...,A bank you can physically go into a building b...
5,1,NaN,NaN,2,0,NaN,1,1,NaN,"having an income and moving away from cash, bu...",...,3,5,1,8,NaN,1,NaN,Youtube and News articles,I like their online banking and also the low r...,a bank is an established institution that can ...
6,1,NaN,NaN,2,0,NaN,1,0,NaN,My parents opened it for me when I was in elem...,...,4,6,7,8,NaN,1,NaN,Family,"I like the low rates, dislike how it is has ta...","venmo is just for transferring money, chase ha..."


In [ ]:
# display encodings for variables
encodings

{'Finished': {'FALSE': 0, 'TRUE': 1},
 'Q4': {'My bank does not offer a mobile app': 0, 'No': 1, 'Yes': 2, nan: 3},
 'Q5': {'No': 0, 'Yes': 1, nan: 2},
 'Q8': {'No': 0, 'Yes': 1, nan: 2},
 'Q9': {'No': 0, 'Yes': 1, nan: 2},
 'Q12': {'No': 0, 'Yes': 1, nan: 2},
 'Q15': {'A few times a year': 0,
  'A few times per month': 1,
  'Almost Never': 2,
  'Once per month': 3,
  'Rarely': 4,
  nan: 5}}

In [ ]:
# concat dummy columns
first_half = pd.concat([data_1, q2_dummies, q3_dummies, q10_dummies, q40_dummies, q16_dummies], axis=1)
first_half.columns

Index(['Finished', 'Q2_10_TEXT', 'Q3_9_TEXT', 'Q4', 'Q5', 'Q6', 'Q8', 'Q9',
       'Q10_4_TEXT', 'Q11', 'Q12', 'Q13', 'Q14_1', 'Q14_2', 'Q14_3', 'Q14_4',
       'Q14_5', 'Q14_6', 'Q14_7', 'Q14_8', 'Q14_8_TEXT', 'Q15', 'Q16_6_TEXT',
       'Q17', 'Q18', 'Q19', 'Q2_Apple Pay', 'Q2_CashApp', 'Q2_Chime',
       'Q2_Google Pay', 'Q2_Mint', 'Q2_PayPal', 'Q2_Robinhood', 'Q2_Venmo',
       'Q2_Zelle', 'Q3_Facebook', 'Q3_Family', 'Q3_Instagram', 'Q3_News',
       'Q3_Reddit', 'Q3_TikTok', 'Q3_X (formerly known as Twitter)',
       'Q3_YouTube', 'Q10_Checking account', 'Q10_Investment portfolio',
       'Q10_Savings account', 'Q40_Automated budgeting tool',
       'Q40_Automatic rounding to savings account', 'Q40_Check credit reports',
       'Q40_I do not use a banking app', 'Q40_Interact with virtual assistant',
       'Q40_Locking cards', 'Q40_Manual budgeting by setting spending limits',
       'Q40_Mobile check deposit', 'Q40_Recurring payments view',
       'Q40_Rewards/Offers', 'Q40_Send 



---

# Cleaning the second half of the data


---




### Cleaning Question 39 - Location

Metropolitan City: This category could be used for the largest cities in the United States, such as New York City, Los Angeles, Chicago, and others. These cities are often part of a metropolitan statistical area (MSA) and have a large population.

Large City: This category could be used for cities with a population of 250,000 or more. These cities are typically part of an urbanized area and are considered large in terms of population size nces.ed.gov.

Midsize City: This category could be used for cities with a population between 100,000 and 250,000. These cities are also part of an urbanized area but are smaller in population size than large cities nces.ed.gov.

Small City: This category could be used for cities with a population less than 100,000. These cities are typically part of an urbanized area and are considered small in terms of population size nces.ed.gov.

Town: This category could be used for smaller communities that are not considered cities. These could be census-designated places (CDPs) or other unincorporated communities en.wikipedia.org.

Rural Area: This category could be used for areas that are not part of an urbanized area. These could be rural areas or nonmetropolitan areas cdc.gov.

In [ ]:
# Seperate by city size
q39 = data_2.loc[:, "Q39"]
q39 = data_2['Q39'].tolist()
city_sizes = [None]*len(q39)

for i in range (len(q39)):
    # Check for no response
    if pd.isnull(q39[i]):
        q39[i] = None
        continue

    # Very large city (pop > 1,000,000)
    vlarge = ['houston', 'new york, new york', 'chicago', 'chigago', 'new york city', 'los angeles', 'newyork', 'phoenix', 'dallas', 'bronx']
    if any(re.search(s, q39[i].lower()) for s in vlarge):
        city_sizes[i] = "very large"
        continue

    # Large city (250,000 < pop < 1,000,000)
    large = ['buffalo', 'milwaukee', 'las vegas', 'denver', 'columbus', 'irvine', 'boston', 'atlanta', 'cleveland', 'shaker heights', 'long beach', 'winston salem']
    if any(re.search(s, q39[i].lower()) for s in large):
        city_sizes[i] = "large"
        continue


    # Midsize city (100,000 < pop < 250,000)
    mid = ['pasadena', 'new haven', 'metairie', 'scottsdale']
    if any(re.search(s, q39[i].lower()) for s in mid):
        city_sizes[i] = "medium"
        continue

    # Small city (pop < 100,000)
    small = ['orrville', 'lakewood', 'stanford', 'lancaster', 'waukegan', 'bremerton', 'butte', 'south jordan', 'vineland', 'bozeman', 'gurnee', 'mclean', 'champaign', 'auburn', 'helena']
    if any(re.search(s, q39[i].lower()) for s in small):
        city_sizes[i] = "small"
        continue

    # Town
    town = ['lula', 'rockledge']
    if any(re.search(s, q39[i].lower()) for s in town):
        city_sizes[i] = "town"
        continue

    # Rura area

    else:
        print(q39[i])


print(city_sizes)

# Insert manual responses
q32id = data_2.columns.get_loc('Q32')
data_2.insert(q32id, 'city_size', city_sizes)
data_2.head(), data_2.shape

United states
City 
New Jersey
State
Unspecified, WA, United States
Illinois
Florida 
Us
Grand Rapids 
California 
美国
美国
诺福克，维吉尼亚州，美国
Florida 
Texas
Clevand,OH,USA
Yorktown,Virginia,US
[None, 'small', 'small', 'very large', 'small', 'very large', None, 'large', 'large', 'large', 'large', 'large', 'large', None, None, 'very large', 'large', 'small', None, 'very large', 'large', 'large', 'large', 'small', None, None, 'very large', None, None, 'very large', 'very large', 'large', 'medium', 'medium', None, 'large', 'small', 'large', 'small', 'large', 'large', 'small', 'small', 'town', 'large', None, 'town', 'very large', None, 'very large', 'very large', None, 'very large', 'very large', 'very large', 'very large', 'very large', 'very large', 'very large', 'large', 'medium', 'small', 'very large', None, 'very large', 'very large', 'very large', 'large', 'very large', 'large', 'large', 'large', 'small', 'large', 'very large', 'large', 'large', None, None, None, None, 'very large', None, 'sm

(                    Q21       Q22       Q23                   Q24  \
 2               Neutral     Agree  Disagree  Not really confident   
 3               Neutral   Neutral  Disagree    Somewhat confident   
 4  Not really confident   Neutral  Disagree    Somewhat confident   
 5    Somewhat confident  Disagree  Disagree  Not really confident   
 6  Not at all confident  Disagree  Disagree  Not at all confident   
 
               Q25                   Q26                 Q39   city_size Q32  \
 2  I do not know.  Not really confident                 NaN        None  19   
 3  I do not know.        Very confident  Champaign, Il, USA       small  20   
 4          Maybe.    Somewhat confident     Gurnee, IL, USA       small  20   
 5  I do not know.               Neutral             Chicago  very large  20   
 6          Maybe.    Somewhat confident      Gurnee, IL USA       small  20   
 
      Q28                    Q29  \
 2  15-17              My family   
 3  15-17              M

In [ ]:
# create manual mappings for variables
ordinal_mapping = {
    'Strongly disagree': 1,
    'Disagree': 2,
    'Neutral': 3,
    'Agree': 4,
    'Strongly agree': 5
}

conf_mapping = {
    'Not at all confident': 1,
    'Not really confident': 2,
    'Neutral': 3,
    'Somewhat confident': 4,
    'Very confident': 5,
}

q25_mapping = {
    'I do not know.': 0,
    'No, I cannot.': 1,
    'Maybe.': 2,
    'Yes, I can.': 3
}

city_size_mapping = {
    'town': 1,
    'small': 2,
    'medium': 3,
    'large': 4,
    'very_large': 5
}

Q32_mapping = {
    '18': 0,
    '19': 1,
    '20': 2,
    '21': 3,
    '22': 4,
    '23': 5,
    '24': 6,
    '25 or older': 7
}

Q28_mapping = {
    'Younger than 15': 1,
    '15-17': 2,
    '18-22': 3,
    '22-25': 4,
}

Q33_mapping = {
    'High school': 1,
    '1-2 years of college': 2,
    '3-4 years of college': 3,
    'A Bachelors Degree': 4,
    'A Master’s degree': 5,
}

# encode variables using manual mappings
ord_data = pd.DataFrame()
ord_data[['Q22', "Q23"]] = data_2[['Q22', "Q23"]].apply(lambda x: x.replace(ordinal_mapping))

ord_data[['Q26', 'Q21', "Q24"]] = data_2[['Q26', 'Q21', "Q24"]].apply(lambda x: x.replace(conf_mapping))

ord_data[['Q25']] = data_2[['Q25']].apply(lambda x: x.replace(q25_mapping))

ord_data[['city_size']] = data_2[['city_size']].apply(lambda x: x.replace(city_size_mapping))

ord_data[['Q32']] = data_2[['Q32']].apply(lambda x: x.replace(Q32_mapping))

ord_data[['Q28']] = data_2[['Q28']].apply(lambda x: x.replace(Q28_mapping))

ord_data[['Q33']] = data_2[['Q33']].apply(lambda x: x.replace(Q33_mapping))

ord_data.head(), ord_data.shape

(   Q22  Q23  Q26  Q21  Q24  Q25   city_size  Q32  Q28  Q33
 2  4.0  2.0  2.0  3.0  2.0  0.0        None  1.0  2.0  2.0
 3  3.0  2.0  5.0  3.0  4.0  0.0           2  2.0  2.0  2.0
 4  3.0  2.0  4.0  2.0  4.0  2.0           2  2.0  2.0  2.0
 5  2.0  2.0  3.0  4.0  2.0  0.0  very large  2.0  2.0  3.0
 6  2.0  2.0  4.0  1.0  1.0  2.0           2  2.0  2.0  2.0,
 (177, 10))

In [ ]:
# dummies for categorical variables
cat_cols = data_2[['Q29', 'Q30', 'Q31', 'Q34']].copy()
cat_data = pd.get_dummies(cat_cols, dummy_na = True)
# combine encoded data with dummy variables
second_half = pd.concat([ord_data, cat_data], axis=1)
second_half.head(), second_half.shape

(   Q22  Q23  Q26  Q21  Q24  Q25   city_size  Q32  Q28  Q33  ...  Q31_>180,000  \
 2  4.0  2.0  2.0  3.0  2.0  0.0        None  1.0  2.0  2.0  ...             0   
 3  3.0  2.0  5.0  3.0  4.0  0.0           2  2.0  2.0  2.0  ...             0   
 4  3.0  2.0  4.0  2.0  4.0  2.0           2  2.0  2.0  2.0  ...             0   
 5  2.0  2.0  3.0  4.0  2.0  0.0  very large  2.0  2.0  3.0  ...             0   
 6  2.0  2.0  4.0  1.0  1.0  2.0           2  2.0  2.0  2.0  ...             1   
 
    Q31_Prefer not to answer  Q31_nan  Q34_Agender  Q34_Cis-man  Q34_Cis-woman  \
 2                         0        0            0            0              1   
 3                         0        0            0            0              1   
 4                         0        0            0            0              1   
 5                         0        0            0            1              0   
 6                         0        0            0            0              1   
 
    Q34_Non-



---

# Combining the cleaned halves of the data

---



In [ ]:
# text questions - 'Q2_10_TEXT', 'Q3_9_TEXT', 'Q6', 'Q10_4_TEXT', 'Q11', 'Q13', ''Q14_8_TEXT', 'Q16_6_TEXT', 'Q17', 'Q18', 'Q19'

# merge data
first_half = first_half.reset_index().drop('index', axis=1)
second_half = second_half.reset_index().drop('index', axis=1)
combineddata = pd.concat([first_half, second_half], axis=1)
combineddata

,Finished,Q2_10_TEXT,Q3_9_TEXT,Q4,Q5,Q6,Q8,Q9,Q10_4_TEXT,Q11,...,"Q31_>180,000",Q31_Prefer not to answer,Q31_nan,Q34_Agender,Q34_Cis-man,Q34_Cis-woman,Q34_Non-binary,Q34_Prefer not to disclose,Q34_Transgender,Q34_nan
0,1,NaN,NaN,2,0,NaN,1,1,NaN,Getting a job and a flow of income,...,0,0,0,0,0,1,0,0,0,0
1,1,NaN,NaN,2,0,NaN,1,1,NaN,Often transferred extra money when needed from...,...,0,0,0,0,0,1,0,0,0,0
2,1,NaN,NaN,2,0,NaN,1,1,NaN,Direct deposit for work,...,0,0,0,0,0,1,0,0,0,0
3,1,NaN,NaN,2,0,NaN,1,1,NaN,"having an income and moving away from cash, bu...",...,0,0,0,0,1,0,0,0,0,0
4,1,NaN,NaN,2,0,NaN,1,0,NaN,My parents opened it for me when I was in elem...,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,1,NaN,ChatGPT,2,0,NaN,1,1,NaN,Work/school,...,0,0,0,0,0,1,0,0,0,0
173,1,NaN,NaN,2,1,Excel,1,1,NaN,I got a job,...,0,0,0,0,0,1,0,0,0,0
174,1,NaN,NaN,2,0,NaN,1,1,NaN,Moved to new country (USA) needed a local bank...,...,0,0,0,0,0,1,0,0,0,0
175,1,NaN,NaN,2,1,Personal Capital,1,1,NaN,Banks provide protection against theft and sec...,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
# generate CSV file for combined data
combineddata.to_csv('/content/drive/Shareddrives/Key Fall 2023/Survey Data/Final code/Clean_Survey_Data.csv')